# Data cleaning & EDA

## 0. Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
from pathlib import Path
import sys

# Add the project root to the Python path to import the modules
project_root = Path().absolute().parent
sys.path.append(str(project_root))

## 1. Pre-cleaning data

In [3]:
# Load and standardize case references
datasets = {
    'referrals': pd.read_csv("../data/raw/BRC-Data/Cases_depersonalised.csv"),
    'hiu': pd.read_csv("../data/raw/BRC-Data/HIU_depersonalised.csv"),
    'snap': pd.read_csv("../data/raw/BRC-Data/SNAP_depersonalised.csv")
}

# Standardize case reference columns
datasets['referrals']['case_ref'] = datasets['referrals']['Case Reference']
datasets['hiu']['case_ref'] = 'CAS-' + datasets['hiu']['Q2.1. CAS-'].astype(str).str.replace('.0', '', regex=False)
datasets['snap']['case_ref'] = datasets['snap']['BRM case number:']

In [4]:
print("=== DUPLICATE ANALYSIS BEFORE CLEANING ===")

for name, df in datasets.items():
    print(f"\n{name.upper()} Dataset:")
    print(f"  Total rows: {len(df):,}")
    print(f"  Unique case_ref: {df['case_ref'].nunique():,}")
    
    # Define columns to exclude from duplication analysis (depersonalized/randomized)
    if name == 'referrals':
        # For referrals: only consider case_ref and referral notes
        analysis_cols = ['case_ref', 'Referral Notes (depersonalised)', 'Referral Date/Time']
        available_cols = [col for col in analysis_cols if col in df.columns]
        print(f"  Analysing duplicates based on: {available_cols}")
    elif name == 'snap':
        # Exclude: Has Disability, IMD Decile, Country, Age, Gender, Ethnicity, Living Arrangements
        exclude_cols = ['Has Disability', 'IMD Decile', 'Country', 'Age', 'Gender', 'Ethnicity', 'Living Arrangements']
        available_cols = [col for col in df.columns if col not in exclude_cols]
        print(f"  Analysing duplicates excluding {len(exclude_cols)} depersonalised columns")
    elif name == 'hiu':
        # Exclude: Age, Gender, Ethnicity, Living Arrangements
        exclude_cols = ['Age', 'Gender', 'Ethnicity', 'Living Arrangements']
        available_cols = [col for col in df.columns if col not in exclude_cols]
        print(f"  Analysing duplicates excluding {len(exclude_cols)} depersonalised columns")
    
    # Check for perfect duplicates based on relevant columns only
    perfect_duplicates = df.duplicated(subset=available_cols).sum()
    print(f"  Perfect duplicates (relevant columns): {perfect_duplicates:,}")
    
    # Check for duplicates by case_ref only
    case_ref_duplicates = df['case_ref'].duplicated().sum()
    print(f"  Duplicate case_ref: {case_ref_duplicates:,}")
    
    if case_ref_duplicates > 0:
        # Show examples of duplicate case_ref: Show top 5 by count
        duplicate_cases = df[df['case_ref'].duplicated(keep=False)]['case_ref'].value_counts().head(5)
        print(f"  Top 5 duplicate case_ref by count:")
        for case_ref, count in duplicate_cases.items():
            print(f"    {case_ref}: {count} rows")
        
        # Check if duplicate case_ref have identical relevant data - CHANGED: Use highest count case
        highest_count_case = duplicate_cases.index[0]  # This is now the case with most duplicates
        duplicate_rows = df[df['case_ref'] == highest_count_case]
        
        # Check duplicates based on relevant columns only
        duplicate_subset = duplicate_rows[available_cols]
        identical_duplicates = duplicate_subset.duplicated().sum()
        
        print(f"    For case {highest_count_case}: {identical_duplicates}/{len(duplicate_rows)-1} duplicates are identical (relevant data)")
        
        # Show what's different in duplicate rows (if any) - only relevant columns
        if identical_duplicates < len(duplicate_rows) - 1:
            print(f"    Non-identical relevant columns for {highest_count_case}:")
            for col in available_cols:
                if col in duplicate_rows.columns:
                    unique_vals = duplicate_rows[col].nunique()
                    if unique_vals > 1:
                        values = duplicate_rows[col].tolist()
                        # For referral notes, show preview
                        if col == 'Referral Notes (depersonalised)':
                            values = [str(v)[:50] + "..." if len(str(v)) > 50 else str(v) for v in values]
                        print(f"      {col}: {values}")

=== DUPLICATE ANALYSIS BEFORE CLEANING ===

REFERRALS Dataset:
  Total rows: 181,085
  Unique case_ref: 126,717
  Analysing duplicates based on: ['case_ref', 'Referral Notes (depersonalised)', 'Referral Date/Time']
  Perfect duplicates (relevant columns): 52,146
  Duplicate case_ref: 54,368
  Top 5 duplicate case_ref by count:
    CAS-537284: 60 rows
    CAS-530729: 45 rows
    CAS-565892: 42 rows
    CAS-592629: 42 rows
    CAS-537229: 40 rows
    For case CAS-537284: 45/59 duplicates are identical (relevant data)
    Non-identical relevant columns for CAS-537284:
      Referral Notes (depersonalised): ['XXXX hazard', '4.Fall', '4.Trips and falls', '4.Trips and falls', 'XXXX hazard', '4.Fall', '4.Trips and falls', 'XXXX hazard', '4.Trips and falls', 'XXXX of benefits', '3.Person who client claimed kidnapped her is now o...', 'Distress', 'XXXX of benefits', 'XXXX hazard', '3.Person who client claimed kidnapped her is now o...', 'Distress', 'abusive behaviour', 'XXXX of benefits', '4.Fa

### 1.1 Pre-cleaning referrals

I start with the pre-cleaning of referrals.

1. I remove identical rows based on: ['case_ref', 'Referral Notes (depersonalised)', 'Referral Date/Time']. Those are pure duplicates (other columns might be different due to the depersonalisation).

Then, for each CAS which has multiple rows (most likely on different dates):

2. I create two columns: one with the number of observations, and one with the date range of those observations;
3. Then, if one or some of the rows have a non-NA referral, I remove the rows which don't. 
4. Finally, if all observations have the same referrals, or if none of the observations have a referral, I only keep the most recent row (the columns created earlier will keep the other relevant information).

The only duplicated CAS left will be those with different referrals on different dates.

In [5]:
from utils.data_cleaning_helpers import clean_referrals_dataset

# Load the referrals data
referrals_df = datasets['referrals'].copy()

# Clean the dataset
referrals_cleaned = clean_referrals_dataset(referrals_df)

=== REFERRALS DATASET PRE-CLEANING ===
Initial dataset: 181,085 rows, 126,717 unique cases

Step 0: Cleaning NA variations in text columns...

Step 1: Removing identical rows...
  Using columns: ['case_ref', 'Referral Notes (depersonalised)', 'Referral Date/Time']
  Removed 52,146 identical rows
  Remaining: 128,939 rows, 126,717 unique cases

Step 2: Processing cases with multiple observations...
  Step 2a: Adding observation count and date range columns...
  Step 2b: Removing rows without referral notes when others exist...
    Removed 0 rows without referral notes
  Step 2c: Consolidating cases with identical or missing referral notes...
    Consolidated 0 rows with identical/missing referral notes

Final dataset: 128,939 rows, 126,717 unique cases

Cases with multiple rows remaining (different referrals): 1,434
Top 5 cases by number of remaining rows:
  CAS-537284: 15 rows
  CAS-530729: 9 rows
  CAS-483850: 8 rows
  CAS-503700: 8 rows
  CAS-560840: 8 rows

Example case CAS-537284 w

In [ ]:
print("=== ANALYSING REFERRALS BY DATE PATTERNS ===")

# Convert date column to datetime if not already
referrals_cleaned['Referral Date/Time'] = pd.to_datetime(referrals_cleaned['Referral Date/Time'], errors='coerce')
referrals_cleaned['referral_date'] = referrals_cleaned['Referral Date/Time'].dt.date

# Cases with multiple referrals
multi_referral_cases = referrals_cleaned['case_ref'].value_counts()
multi_referral_cases = multi_referral_cases[multi_referral_cases > 1]

print(f"Cases with multiple referrals: {len(multi_referral_cases):,}")
print(f"Cases with single referral: {referrals_cleaned['case_ref'].nunique() - len(multi_referral_cases):,}")

# Analyze date patterns for multi-referral cases
print(f"\n=== DATE PATTERNS FOR {len(multi_referral_cases):,} MULTI-REFERRAL CASES ===")

same_date_stats = []
for case_ref in multi_referral_cases.index:
    case_data = referrals_cleaned[referrals_cleaned['case_ref'] == case_ref].copy()
    
    # Count unique dates for this case
    unique_dates = case_data['referral_date'].nunique()
    total_referrals = len(case_data)
    
    same_date_stats.append({
        'case_ref': case_ref,
        'total_referrals': total_referrals,
        'unique_dates': unique_dates,
        'all_same_date': unique_dates == 1,
        'multiple_dates': unique_dates > 1
    })

same_date_df = pd.DataFrame(same_date_stats)

# Summary statistics
all_same_date = same_date_df['all_same_date'].sum()
multiple_dates = same_date_df['multiple_dates'].sum()

print(f"Cases where ALL referrals are on the SAME date: {all_same_date:,} ({all_same_date/len(multi_referral_cases)*100:.1f}%)")
print(f"Cases with referrals on MULTIPLE dates: {multiple_dates:,} ({multiple_dates/len(multi_referral_cases)*100:.1f}%)")

=== ANALYZING REFERRALS BY DATE PATTERNS ===
Cases with multiple referrals: 1,434
Cases with single referral: 125,283

=== DATE PATTERNS FOR 1,434 MULTI-REFERRAL CASES ===
Cases where ALL referrals are on the SAME date: 1,434 (100.0%)
Cases with referrals on MULTIPLE dates: 0 (0.0%)


For cases which have multiple observations with DIFFERENT referrals, they are always done on the same date. I decide to keep the longest referral, to keep the maximum information.

In [9]:
from utils.data_cleaning_helpers import consolidate_referrals_longest

# Apply consolidation
referrals_consolidated = consolidate_referrals_longest(referrals_cleaned)

# Save consolidated dataset
referrals_consolidated.to_csv("../data/processed/referrals_cleaned.csv", index=False)
print(f"\nConsolidated referrals saved to: ../data/processed/referrals_cleaned.csv")

=== CONSOLIDATING REFERRALS TO ONE PER CASE ===
Initial referrals: 128,939 rows, 126,717 unique cases
Cases with multiple referrals: 1,434
After consolidation: 126,717 rows (one per case)

Consolidated referrals saved to: ../data/processed/referrals_cleaned.csv


### 1.2 Pre-cleaning SNAP data & merging with referrals

I start with **SNAP** (Support at Home, Care at Home, Hospital at Home, and Social Prescribing services).

These services have started using an outcomes framework called SNAP (Social Needs and Preferences), which was only introduced in late 2024.

The pre-cleaning of SNAP is done as follows:

1. I removed perfect duplicates based on columns that were not randomised in the depersonalisation process.

Then, I noticed that CAS have either one or two observations (not more). These correspond to observations at the start and / or at the end of support from the BRC. I get a brief overview.

2. For each case, I determine what type of valid assessments are available, only counting valid where 'Possible to record outcomes:' == 'Yes'. 
3. I then create a summary for each case showing:
    - Total number of assessments
    - Whether valid baseline assessment exists (timepoint 1.0 + recordable outcomes)
    - Whether valid post-support assessment exists (timepoint 2.0 + recordable outcomes)
    - Whether case has both valid assessments (complete usable pair)

In [17]:
from utils.data_cleaning_helpers import clean_snap_dataset

snap_df = datasets['snap'].copy()

# Clean the SNAP dataset
snap_cleaned = clean_snap_dataset(snap_df)

# Save cleaned dataset
snap_cleaned.to_csv("../data/processed/snap_cleaned.csv", index=False)
print(f"\nCleaned dataset saved to: ../data/processed/snap_cleaned.csv")


=== SNAP DATASET PRE-CLEANING ===
Initial dataset: 2,012 rows, 1,305 unique cases

Step 0: Cleaning NA variations in text columns...

Step 1: Removing perfect duplicates...
  Removed 54 perfect duplicates

Step 2: Identifying valid baseline and valid post-support assessments...
  Cases with valid baseline: 983
  Cases with valid baseline only: 508
  Cases with valid post-support: 540

  Cases with both (complete valid pairs): 475

Step 3: Adding metadata columns...

Final dataset: 1,958 rows, 1,305 unique cases

Cleaned dataset saved to: ../data/processed/snap_cleaned.csv


**Merging with referrals**

I now proceed to creating a merged dataset with referrals.

In [11]:
def merge_referrals_with_snap():
    """
    Merge referrals (1 row per case) with SNAP data (1-2 rows per case).
    Only keeps cases that appear in both datasets.
    """
    
    print("=== MERGING REFERRALS WITH SNAP DATA ===")
    
    # Load datasets
    referrals_df = pd.read_csv("../data/processed/referrals_cleaned.csv")
    snap_df = pd.read_csv("../data/processed/snap_cleaned.csv")
    
    print(f"Referrals dataset: {len(referrals_df):,} rows, {referrals_df['case_ref'].nunique():,} unique cases")
    print(f"SNAP dataset: {len(snap_df):,} rows, {snap_df['case_ref'].nunique():,} unique cases")
    
    # Find cases that appear in both datasets
    referrals_cases = set(referrals_df['case_ref'])
    snap_cases = set(snap_df['case_ref'])
    
    common_cases = referrals_cases.intersection(snap_cases)
    
    print(f"\nCases in both datasets: {len(common_cases):,}")
    print(f"Cases only in referrals: {len(referrals_cases - snap_cases):,}")
    print(f"Cases only in SNAP: {len(snap_cases - referrals_cases):,}")
    
    # Filter both datasets to common cases only
    referrals_common = referrals_df[referrals_df['case_ref'].isin(common_cases)].copy()
    snap_common = snap_df[snap_df['case_ref'].isin(common_cases)].copy()
    
    print(f"\nAfter filtering to common cases:")
    print(f"Referrals: {len(referrals_common):,} rows")
    print(f"SNAP: {len(snap_common):,} rows")
    
    # Merge: each referral row will be duplicated for each SNAP row of the same case
    merged_df = snap_common.merge(referrals_common, on='case_ref', how='inner', suffixes=('_snap', '_referral'))
    
    print(f"\nMerged dataset: {len(merged_df):,} rows, {merged_df['case_ref'].nunique():,} unique cases")
    
    # Show structure breakdown
    rows_per_case = merged_df['case_ref'].value_counts()
    print(f"\nRows per case in merged dataset:")
    print(f"  1 row (baseline only): {(rows_per_case == 1).sum():,} cases")
    print(f"  2 rows (baseline + outcome): {(rows_per_case == 2).sum():,} cases")
    if (rows_per_case > 2).any():
        print(f"  >2 rows (unexpected): {(rows_per_case > 2).sum():,} cases")
    
    # Show SNAP assessment validity breakdown for merged cases
    if 'has_both' in merged_df.columns:
        cases_with_both = merged_df.drop_duplicates('case_ref')['has_both'].sum()
        cases_baseline_only = merged_df.drop_duplicates('case_ref')['has_valid_baseline'].sum() - cases_with_both
        
        print(f"\nSNAP assessment validity in merged dataset:")
        print(f"  Cases with valid baseline + outcome: {cases_with_both:,}")
        print(f"  Cases with valid baseline only: {cases_baseline_only:,}")
    
    return merged_df

# Perform the merge
merged_referrals_snap = merge_referrals_with_snap()

# Save merged dataset
merged_referrals_snap.to_csv("../data/processed/merged_referrals_snap.csv", index=False)
print(f"\nMerged dataset saved to: ../data/processed/merged_referrals_snap.csv")

=== MERGING REFERRALS WITH SNAP DATA ===
Referrals dataset: 126,717 rows, 126,717 unique cases
SNAP dataset: 1,958 rows, 1,305 unique cases

Cases in both datasets: 1,222
Cases only in referrals: 125,495
Cases only in SNAP: 84

After filtering to common cases:
Referrals: 1,222 rows
SNAP: 1,856 rows

Merged dataset: 1,856 rows, 1,222 unique cases

Rows per case in merged dataset:
  1 row (baseline only): 588 cases
  2 rows (baseline + outcome): 634 cases

SNAP assessment validity in merged dataset:
  Cases with valid baseline + outcome: 472
  Cases with valid baseline only: 471

Merged dataset saved to: ../data/processed/merged_referrals_snap.csv


Let's now check the completeness of the data for cases which have both baseline and post-support observations.

In [14]:
# Data completeness for key domains
key_domains = ['Control (QLC)', 'Personal cleanliness (QLC)', 'Food and drink (QLC)', 
               'Personal safety (QLC)', 'Social Participation (QLC)', 'Occupation (QLC)', 
               'Accommodation (QLC)', 'Dignity 1 (QLC)', 'snap_Medication (QLC)', 'Finances (QLC)']

# Filter rows with both baseline and outcome
filtered_snap = merged_referrals_snap[merged_referrals_snap['has_both']]

print(f"\n=== DATA COMPLETENESS ===")
for domain in key_domains:
    if domain in filtered_snap.columns:
        completeness = filtered_snap[domain].notna().mean()
        print(f"{domain}: {completeness:.1%} complete")



=== DATA COMPLETENESS ===
Control (QLC): 91.0% complete
Personal cleanliness (QLC): 91.8% complete
Food and drink (QLC): 92.2% complete
Personal safety (QLC): 93.0% complete
Social Participation (QLC): 90.1% complete
Occupation (QLC): 90.5% complete
Accommodation (QLC): 91.6% complete
Dignity 1 (QLC): 90.4% complete
Finances (QLC): 86.7% complete


### 1.3 Pre-cleaning HIU data & merging with referrals

## 2. Exploratory Data Analysis

In [8]:
from utils.data_cleaning_helpers import get_note_length_category

# Import referrals dataset
referrals_df = pd.read_csv("../data/processed/referrals_cleaned.csv")

# Add note categories
referrals_df['note_length_category'] = referrals_df['Referral Notes (depersonalised)'].apply(get_note_length_category)

In [9]:
note_counts = referrals_df['note_length_category'].value_counts()

print(f"\n=== REFERRAL NOTES QUALITY ===")
for category, count in note_counts.items():
    print(f"{category}: {count:,} ({count/len(referrals_df):.1%})")


=== REFERRAL NOTES QUALITY ===
Short note (<5 words): 47,735 (37.7%)
Medium note (5-19 words): 31,272 (24.7%)
No note: 25,237 (19.9%)
Long note (20+ words): 22,473 (17.7%)


### HIU (High Intensity Use service for people frequently attending A&E)

In [28]:
# Filter dataset to only include referrals with HIU data
merged_hiu = merged[merged['has_hiu']].copy()
merged_hiu = merged_hiu.loc[:, ~merged_hiu.columns.str.startswith('snap_')]

cols_hiu = list(merged_hiu.columns)

In [33]:
print("=== HIU DATA OVERVIEW ===")
print(f"Total HIU observations: {len(merged_hiu):,}")
print(f"Unique cases: {merged_hiu['case_ref'].nunique():,}")

# Check for exact duplicates
print(f"\n=== DUPLICATE ANALYSIS ===")
print(f"Total rows: {len(merged_hiu):,}")

# Check duplicates based on case reference and all HIU variables
hiu_cols = [col for col in merged_hiu.columns if col.startswith('hiu_')]
duplicate_subset = ['case_ref'] + hiu_cols

duplicates = merged_hiu.duplicated(subset=duplicate_subset, keep=False)
print(f"Exact duplicate rows (case + HIU variables): {duplicates.sum():,}")

if duplicates.sum() > 0:
    print(f"\nExample of duplicate cases:")
    duplicate_cases = merged_hiu[duplicates]['case_ref'].value_counts().head(3)
    for case_ref, count in duplicate_cases.items():
        print(f"  {case_ref}: {count} identical rows")

# Remove exact duplicates
print(f"\n=== REMOVING DUPLICATES ===")
merged_hiu_clean = merged_hiu.drop_duplicates(subset=duplicate_subset, keep='first')
print(f"After removing duplicates: {len(merged_hiu_clean):,} rows")
print(f"Duplicates removed: {len(merged_hiu) - len(merged_hiu_clean):,}")
print(f"Unique cases after cleaning: {merged_hiu_clean['case_ref'].nunique():,}")

=== HIU DATA OVERVIEW ===
Total HIU observations: 4,269
Unique cases: 1,400

=== DUPLICATE ANALYSIS ===
Total rows: 4,269
Exact duplicate rows (case + HIU variables): 2,932

Example of duplicate cases:
  CAS-525447: 27 identical rows
  CAS-527026: 24 identical rows
  CAS-482848: 20 identical rows

=== REMOVING DUPLICATES ===
After removing duplicates: 2,370 rows
Duplicates removed: 1,899
Unique cases after cleaning: 1,400
